In [1]:
#First, the point cloud has to be loaded and then read.
#Its probably way too big to work with from the start, so I think I will use voxel downsampling
#But first, imports

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os   


#Loading the point cloud

initialCloud =  o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "croppedCloud.ply")) 




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
o3d.visualization.draw_geometries([initialCloud])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 


In [3]:
#Making a voxelgrid from the point cloud
voxel_size = 1
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(initialCloud, voxel_size)



In [4]:
#Extracting center points so we can calculate PCA for the points within each voxel

voxel_center_coordinates= []

for i in voxel_grid.get_voxels():
    voxel_center_coordinates.append(voxel_grid.get_voxel_center_coordinate(i.grid_index))



In [ ]:
#This is old code, not if needed anymore
from sklearn.decomposition import PCA
from tqdm.auto import tqdm

#Extracting points from each voxel, calculating features for all of them and storing them in lists
voxel_size = 3


linearity =[]
planarity = []
scattering = []
omnivariance = []
sum_eigenvalues = []
anisotropy = []
eigentropy = []
change_of_curvature = []
points_per_voxel = []
z_range = []
 

#Calculating Z-range and standard deviation from the plane for each voxel

#Z-range

#Defining a function to calculate the z range
def calculateZRange(points):
    if len(points) < 15:
        return 0
    return(np.max(points[:,2]) - np.min(points[:,2]))
   
#Run this to just run it on the first 100 voxels
#voxel_center_coordinates = voxel_center_coordinates[:100]


for center in tqdm(voxel_center_coordinates):
    minboundx = center[0] - voxel_size/2
    minboundy = center[1] - voxel_size/2
    minboundz = center[2] - voxel_size/2

    maxboundx = center[0] + voxel_size/2
    maxboundy = center[1] + voxel_size/2
    maxboundz = center[2] + voxel_size/2

    voxel_extracted_points =initialCloud.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(minboundx, minboundy, minboundz), max_bound=(maxboundx, maxboundy, maxboundz)))
    #print("Number of points in voxel ", len(voxel_extracted_points.points))
    
    points = np.asarray(voxel_extracted_points.points)
    
    #Checking if there are enough points in the voxel to calculate PCA
    
    if len(points) < 15:
        linearity.append(0)
        planarity.append(0)
        scattering.append(0)
        omnivariance.append(0)
        sum_eigenvalues.append(0)
        anisotropy.append(0)
        eigentropy.append(0)
        change_of_curvature.append(0)
        points_per_voxel.append(points) #Kan her appende 0 istedenfor points?
        continue
    
    pca = PCA(n_components=3)
    pca.fit(points)

    # Extract the eigenvalues and eigenvectors
    eigenvalues = pca.explained_variance_
    eigenvectors = pca.components_
    
    #Calculating the features
    l = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0] 
    p = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
    s = eigenvalues[2] / eigenvalues[0]
    o = (eigenvalues[0]*eigenvalues[1]*eigenvalues[2])**(1/3)
    s_e = eigenvalues[0] + eigenvalues[1] + eigenvalues[2]
    a = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
    e = (-1)*(eigenvalues[0]*np.log(eigenvalues[0]) + eigenvalues[1]*np.log(eigenvalues[1]) + eigenvalues[2]*np.log(eigenvalues[2]))
    c_o_c = eigenvalues[2]/(eigenvalues[0]+eigenvalues[1]+eigenvalues[2])
    
    
   
    

In [6]:
#Defining a function to extract points from a voxel based on center coordinates. Returns a list of points

from sklearn.decomposition import PCA
from tqdm.auto import tqdm


def extractPointCloudFromVoxel(voxel_center_coordinates, pointCloud, voxel_size):
    extractedPointClouds = []
    for center in tqdm(voxel_center_coordinates):
        minboundx = center[0] - voxel_size/2
        minboundy = center[1] - voxel_size/2
        minboundz = center[2] - voxel_size/2

        maxboundx = center[0] + voxel_size/2
        maxboundy = center[1] + voxel_size/2
        maxboundz = center[2] + voxel_size/2

        pcd = pointCloud.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(minboundx, minboundy, minboundz), max_bound=(maxboundx, maxboundy, maxboundz)))
        # Voxels with less than 10 points are ignored and not appended
        
        if len(pcd.points) > 10:
            extractedPointClouds.append(pcd)
    return extractedPointClouds
    
    
    

In [7]:
#Function to calculate features from a list of points clouds (could be one or more), returns lists for each feature
def calculatePCAFeatures(listOfPointClouds):
    
    linearity =[]
    planarity = []
    scattering = []
    omnivariance = []
    sum_eigenvalues = []
    anisotropy = []
    eigentropy = []
    change_of_curvature = []
    eigenvaluesList = []
    eigenvectorsList = []
    points_per_voxel = []
    
    for pcd in tqdm(listOfPointClouds):
        points = pcd.points
    
        '''
        if len(points) < 15:
            linearity.append(0)
            planarity.append(0)
            scattering.append(0)
            omnivariance.append(0)
            sum_eigenvalues.append(0)
            anisotropy.append(0)
            eigentropy.append(0)
            change_of_curvature.append(0)
            points_per_voxel.append(points) #Kan her appende 0 istedenfor points?
            continue
            ''' #Denne sjekken blir gjort i extractPointCloudFromVoxel, slik at man slipper å gjøre det her
    
        pca = PCA(n_components=3)
        pca.fit(points)

    # Extract the eigenvalues and eigenvectors
        eigenvalues = pca.explained_variance_
        eigenvectors = pca.components_
    
      #Calculating the features
        l = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0] 
        p = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
        s = eigenvalues[2] / eigenvalues[0]
        o = (eigenvalues[0]*eigenvalues[1]*eigenvalues[2])**(1/3)
        s_e = eigenvalues[0] + eigenvalues[1] + eigenvalues[2]
        a = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
        e = (-1)*(eigenvalues[0]*np.log(eigenvalues[0]) + eigenvalues[1]*np.log(eigenvalues[1]) + eigenvalues[2]*np.log(eigenvalues[2]))
        c_o_c = eigenvalues[2]/(eigenvalues[0]+eigenvalues[1]+eigenvalues[2])

        linearity.append(l)
        planarity.append(p)
        scattering.append(s)
        omnivariance.append(o)
        sum_eigenvalues.append(s_e)
        anisotropy.append(a)
        eigentropy.append(e)
        change_of_curvature.append(c_o_c)
        eigenvaluesList.append(eigenvalues)
        eigenvectorsList.append(eigenvectors)
        points_per_voxel.append(points)
        
    return linearity, planarity, scattering, omnivariance, sum_eigenvalues, anisotropy, eigentropy, change_of_curvature, eigenvaluesList, eigenvectorsList, points_per_voxel

In [37]:
for i in points_per_voxel:
    print(i)

std::vector<Eigen::Vector3d> with 101 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 453 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 598 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 168 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 85 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 344 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 43 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 284 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 63 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 50 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 346 elements.
Use numpy.asarray() to access data.
std::vector<Eigen::Vector3d> with 541 elements.
Use numpy.asarray() to access da

In [8]:
#Defining a seperate function to calculate z-range for a set of points
def calculateZRange(pointClouds):
    z_range = []
    for pcd in tqdm(pointClouds):
        points = np.asarray(pcd.points)
        '''
        if len(points) < 15:
            z_range.append(0)
            continue
            ''' #Denne sjekken blir gjort i extractPointCloudFromVoxel
        z = np.max(points[:,2]) - np.min(points[:,2])
        z_range.append(z)
    return z_range
   

In [9]:
#Extracting the points from each voxel

extractedPointClouds = extractPointCloudFromVoxel(voxel_center_coordinates, initialCloud, voxel_size)

  0%|          | 0/268674 [00:00<?, ?it/s]

KeyboardInterrupt: 

std::vector<Eigen::Vector3d> with 101 elements.
Use numpy.asarray() to access data.


In [32]:
print(extractedPointClouds[1])

PointCloud with 453 points.


In [9]:
#Calculating the features for each voxel
linearity, planarity, scattering, omnivariance, sum_eigenvalues, anisotropy, eigentropy, change_of_curvature, eigenvalues, eigenvectors, points_per_voxel = calculatePCAFeatures(extractedPointClouds)

  0%|          | 0/25009 [00:00<?, ?it/s]

In [10]:
#Calculating z-range for each voxel IT WORKS

z_range = calculateZRange(extractedPointClouds)


  0%|          | 0/25009 [00:00<?, ?it/s]

In [11]:
#Making a dataframe with all the features and the points for each voxel with an empty label column

import pandas as pd

# both lists, with columns specified
label = [0] * len(linearity) #Placeholder zeroes

df = pd.DataFrame(list(zip(points_per_voxel, linearity, planarity, scattering, omnivariance, sum_eigenvalues, anisotropy, eigentropy, change_of_curvature, z_range, label, )),
			columns =[ 'points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'z_range', 'label', ])



,points,linearity,planarity,scattering,omnivariance,sum_eigenvalues,anisotropy,eigentropy,change_of_curvature,z_range,label
0,"[[570149.970001, 7034352.429993, 4.96], [57015...",0.439892,0.085995,0.474113,0.581093,1.839046,0.525887,0.798887,0.233069,2.760000,0.0
1,"[[570268.780029, 7034220.920013, 27.32], [5702...",0.429511,0.557233,0.013256,0.167289,1.349800,0.986744,0.537485,0.008370,0.490000,0.0
2,"[[570331.419983, 7034335.01001, 20.65], [57033...",0.164252,0.835163,0.000585,0.064187,1.496035,0.999415,0.432333,0.000319,0.530001,0.0
3,"[[570203.660004, 7034216.220001, 24.42], [5702...",0.705547,0.261180,0.033273,0.208933,1.296424,0.966727,0.493010,0.025060,1.090000,0.0
4,"[[570257.23999, 7034343.890015, 16.940001], [5...",0.501917,0.370362,0.127722,0.107745,0.438810,0.872278,0.739346,0.078559,1.390000,1.0
...,...,...,...,...,...,...,...,...,...,...,...
25004,"[[570170.399994, 7034178.299988, 28.870001], [...",0.992355,0.007010,0.000635,0.012349,0.735334,0.999365,0.262710,0.000630,0.149999,2.0
25005,"[[570142.23999, 7034172.929993, 31.870001], [5...",0.885134,0.098229,0.016638,0.095079,0.866918,0.983362,0.473569,0.014704,1.300000,1.0
25006,"[[570142.279999, 7034175.869995, 31.889999], [...",0.803574,0.189768,0.006658,0.095638,1.052161,0.993342,0.449791,0.005534,1.310001,1.0
25007,"[[570107.570007, 7034347.059998, 8.07], [57010...",0.870131,0.046888,0.082982,0.028768,0.157965,0.917018,0.383417,0.068419,0.530000,0.0


In [28]:
df.to_csv(os.path.join(os.getcwd(), "data", "voxelFeatures.csv")) #Saving the dataframe to a csv file, overwrite later with 1 meter voxel

In [14]:
import os
#Read all the houses into a list

houses_folder = os.path.join(os.getcwd(), "data", "Houses")
houses_files = os.listdir(houses_folder)

houses = []
for file in houses_files:
    if file.endswith(".ply"):
        house = o3d.io.read_point_cloud(os.path.join(houses_folder, file))
        houses.append(house)


In [15]:
#Load all the trees into a list

trees_folder = os.path.join(os.getcwd(), "data", "Trees")   
trees_files = os.listdir(trees_folder)

trees = []
for file in trees_files:
    if file.endswith(".ply"):
        tree = o3d.io.read_point_cloud(os.path.join(trees_folder, file))
        trees.append(tree)


In [16]:
#Load all terrain into a list

terrain_folder = os.path.join(os.getcwd(), "data", "Terrain")
terrain_files = os.listdir(terrain_folder)

terrain = []
for file in terrain_files:
    if file.endswith(".ply"):
        terrain_piece = o3d.io.read_point_cloud(os.path.join(terrain_folder, file))
        terrain.append(terrain_piece)


In [17]:
#Parameters for houses

#First making a point cloud of all houses
houseCloud = o3d.geometry.PointCloud()
for house in houses:
    houseCloud+= house
    
#Making a voxel grid of the houses
houseVoxelGrid = o3d.geometry.VoxelGrid.create_from_point_cloud(houseCloud, voxel_size)

#Extracting center points
houseVoxelCenterCoordinates= []
for i in houseVoxelGrid.get_voxels():
    houseVoxelCenterCoordinates.append(houseVoxelGrid.get_voxel_center_coordinate(i.grid_index))
    
#Extracting points from each voxel
houseExtractedPointClouds = extractPointCloudFromVoxel(houseVoxelCenterCoordinates, houseCloud, voxel_size)

#Calculating the PCA features for each voxel
houseLinearity, housePlanarity, houseScattering, houseOmnivariance, houseSum_eigenvalues, houseAnisotropy, houseEigentropy, houseChange_of_curvature, houseEigenvalues, houseEigenvectors, housePoints_per_voxel = calculatePCAFeatures(houseExtractedPointClouds)

#Calculating other features
houseZ_range = calculateZRange(houseExtractedPointClouds)

#Making an array of labels for the houses

houseLabels = np.ones(len(houseLinearity))

#Making a dataframe with all the features and the points for each voxel
dfHouse = pd.DataFrame(list(zip(housePoints_per_voxel, houseLinearity, housePlanarity, houseScattering, houseOmnivariance, houseSum_eigenvalues, houseAnisotropy, houseEigentropy, houseChange_of_curvature, houseZ_range, houseLabels)),
                       columns = ['points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'z_range', 'label'])

  0%|          | 0/2991 [00:00<?, ?it/s]

  0%|          | 0/2446 [00:00<?, ?it/s]

  0%|          | 0/2446 [00:00<?, ?it/s]

In [18]:
#Parameters for trees

#First making a point cloud of all trees
treeCloud = o3d.geometry.PointCloud()
for tree in trees:
    treeCloud+= tree
    
#Making a voxel grid of the trees
treeVoxelGrid = o3d.geometry.VoxelGrid.create_from_point_cloud(treeCloud, voxel_size)

#Extracting center points
treeVoxelCenterCoordinates= []
for i in treeVoxelGrid.get_voxels():
    treeVoxelCenterCoordinates.append(treeVoxelGrid.get_voxel_center_coordinate(i.grid_index))

#Extracting points from each voxel
treeExtractedPointClouds = extractPointCloudFromVoxel(treeVoxelCenterCoordinates, treeCloud, voxel_size)

#Calculating the PCA features for each voxel
treeLinearity, treePlanarity, treeScattering, treeOmnivariance, treeSum_eigenvalues, treeAnisotropy, treeEigentropy, treeChange_of_curvature, treeEigenvalues, treeEigenvectors, treePoints_per_voxel = calculatePCAFeatures(treeExtractedPointClouds)

#Calculating other features
treeZ_range = calculateZRange(treeExtractedPointClouds)

#Making an array of labels for the trees
treeLabels = np.zeros(len(treeLinearity))   

#Making a dataframe with all the features and the points for each voxel
dfTree = pd.DataFrame(list(zip(treePoints_per_voxel, treeLinearity, treePlanarity, treeScattering, treeOmnivariance, treeSum_eigenvalues, treeAnisotropy, treeEigentropy, treeChange_of_curvature, treeZ_range, treeLabels)),
                       columns = ['points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'z_range', 'label'])



  0%|          | 0/4342 [00:00<?, ?it/s]

  0%|          | 0/3373 [00:00<?, ?it/s]

  0%|          | 0/3373 [00:00<?, ?it/s]

In [19]:
#Parameters for terrain

#First making a point cloud of all terrain
terrainCloud = o3d.geometry.PointCloud()

for terrain_piece in terrain:
    terrainCloud+= terrain_piece

#Making a voxel grid of the terrain
terrainVoxelGrid = o3d.geometry.VoxelGrid.create_from_point_cloud(terrainCloud, voxel_size)

#Extracting center points
terrainVoxelCenterCoordinates= []
for i in terrainVoxelGrid.get_voxels():
    terrainVoxelCenterCoordinates.append(terrainVoxelGrid.get_voxel_center_coordinate(i.grid_index))

#Extracting points from each voxel
terrainExtractedPointClouds = extractPointCloudFromVoxel(terrainVoxelCenterCoordinates, terrainCloud, voxel_size)

#Calculating the PCA features for each voxel
terrainLinearity, terrainPlanarity, terrainScattering, terrainOmnivariance, terrainSum_eigenvalues, terrainAnisotropy, terrainEigentropy, terrainChange_of_curvature, terrainEigenvalues, terrainEigenvectors, terrainPoints_per_voxel = calculatePCAFeatures(terrainExtractedPointClouds)

#Calculating other features
terrainZ_range = calculateZRange(terrainExtractedPointClouds)

#Making an array of labels for the terrain with number 2
terrainLabels = np.full(len(terrainLinearity), 2)

#Making a dataframe with all the features and the points for each voxel
dfTerrain = pd.DataFrame(list(zip(terrainPoints_per_voxel, terrainLinearity, terrainPlanarity, terrainScattering, terrainOmnivariance, terrainSum_eigenvalues, terrainAnisotropy, terrainEigentropy, terrainChange_of_curvature, terrainZ_range, terrainLabels)),
                       columns = ['points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'z_range', 'label'])




  0%|          | 0/1971 [00:00<?, ?it/s]

  0%|          | 0/1724 [00:00<?, ?it/s]

  0%|          | 0/1724 [00:00<?, ?it/s]

In [20]:
#Now its time for the machine learning part

#First, we need to split the data into training and testing data, and also try to drop some features to see if it improves the accuracy

from sklearn.model_selection import train_test_split

#Combining the datasets into one dataframe
dfCombined = pd.concat([dfHouse, dfTree, dfTerrain], ignore_index=True)


#Placing the labels in a dataframe
y_train = dfCombined['label']

#Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfCombined.drop(['points', 'label',], axis=1), dfCombined['label'], test_size=0.20, random_state=42)




In [21]:
#Training a decision tree classifier

from sklearn.tree import DecisionTreeClassifier

#Creating the classifier
clf = DecisionTreeClassifier(random_state=0)

#Training the classifier
clf.fit(X_train, y_train)


#Testing it
y_pred = clf.predict(X_test)



In [ ]:
#Going to try autoML to see if it can find a better model




In [22]:
#Testing the accuracy of the classifier

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

feature_importances = clf.feature_importances_
print(feature_importances)



0.6978131212723658
[0.04558361 0.09685715 0.03610405 0.07778357 0.07635215 0.04414846
 0.07865943 0.36538237 0.17912921]


In [23]:
#Now actually testing it on the point cloud

#We already have the dataframe for the entire point cloud, so we can just use that
#We need to drop the points column and the label column
X_cloud = df.drop(['points', 'label'], axis=1)

#Predicting the labels for the point cloud
y_cloud_pred = clf.predict(X_cloud)

#Adding the labels to the dataframe
df['label'] = y_cloud_pred

In [24]:
df.describe()

,linearity,planarity,scattering,omnivariance,sum_eigenvalues,anisotropy,eigentropy,change_of_curvature,z_range,label
count,25009.000000,25009.000000,25009.000000,25009.000000,25009.000000,25009.000000,25009.000000,25009.000000,25009.000000,25009.000000
mean,0.477179,0.426769,0.096052,0.239532,1.292092,0.903948,0.510171,0.054107,1.733100,0.672518
std,0.259898,0.261937,0.118426,0.181182,0.581260,0.118426,0.347132,0.058148,0.973220,0.735543
min,0.001490,0.000229,0.000029,0.003861,0.029873,0.137401,-5.789395,0.000023,0.010000,0.000000
25%,0.262766,0.210512,0.008820,0.084763,0.895836,0.860357,0.426567,0.006029,0.840000,0.000000
50%,0.486231,0.388787,0.050099,0.196575,1.382181,0.949901,0.557967,0.033563,1.780001,1.000000
75%,0.683719,0.617977,0.139643,0.359871,1.649044,0.991180,0.693984,0.085183,2.730000,1.000000
max,0.998379,0.997776,0.862599,1.149165,4.551829,0.999971,1.083444,0.304560,3.000000,2.000000


In [25]:
#Now we need to extract the points from the trees and houses

#Making a dataframe for trees and houses
dfTrees = df[df['label'] == 0]
dfHouses = df[df['label'] == 1]
dfTerrain = df[df['label'] == 2]

#Extracting the point-clusters from the trees
treeClusters= dfTrees['points']
houseClusters = dfHouses['points']
terrainClusters = dfTerrain['points']

classifiedTreeCloud = o3d.geometry.PointCloud()
classifiedHouseCloud = o3d.geometry.PointCloud()
classifiedTerrainCloud = o3d.geometry.PointCloud()

#Painting them

#Trees get green
for cluster in treeClusters:
    cld = o3d.geometry.PointCloud(cluster)
    cld.paint_uniform_color([0, 1, 0])  
    classifiedTreeCloud += cld

#Houses get red
for cluster in houseClusters:
    cld = o3d.geometry.PointCloud(cluster)
    cld.paint_uniform_color([1, 0, 0])  
    classifiedHouseCloud += cld
    
#Terrain gets yellow
for cluster in terrainClusters:
    cld = o3d.geometry.PointCloud(cluster)
    cld.paint_uniform_color([0, 0, 1])
    classifiedTerrainCloud += cld


In [ ]:
newCloud = o

In [39]:
houseClusters

4        [[570257.23999, 7034343.890015, 16.940001], [5...
6        [[570254.919983, 7034328.700012, 18.379999], [...
7        [[570290.76001, 7034338.099976, 30.75], [57029...
8        [[570348.02002, 7034382.76001, 26.16], [570348...
10       [[570272.47998, 7034228.209991, 31.540001], [5...
                               ...                        
25002    [[570142.369995, 7034176.019989, 27.92], [5701...
25003    [[570155.929993, 7034175.940002, 28.77], [5701...
25005    [[570142.23999, 7034172.929993, 31.870001], [5...
25006    [[570142.279999, 7034175.869995, 31.889999], [...
25008    [[570156.070007, 7034309.209991, 7.05], [57015...
Name: points, Length: 8797, dtype: object

In [34]:
o3d.visualization.draw_geometries([classifiedHouseCloud, classifiedTreeCloud, classifiedTerrainCloud])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
